这个是知道 位置信息，知道深度图还有二维图像画面，重构3D点云图

In [1]:
import numpy as np
import cv2
import open3d as o3d
from scipy.spatial.transform import Rotation as R

def read_pose_file(filename):
    poses = []
    with open(filename, 'r') as f:
        for line in f:
            data = list(map(float, line.strip().split()))
            q = data[3:]  # 四元数部分
            t = data[:3]  # 平移向量部分
            
            # 创建变换矩阵
            T = np.eye(4)
            T[:3, :3] = R.from_quat(q).as_matrix()
            T[:3, 3] = t
            
            poses.append(T)
    return poses


read_pose_file("pose.txt")

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[array([[ 0.97226635,  0.06500952, -0.22465952, -0.228993  ],
        [-0.06481371,  0.99786324,  0.00825435,  0.00645704],
        [ 0.22471608,  0.00653559,  0.97440236,  0.0287837 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.77722853,  0.14876417, -0.61137961, -0.50237   ],
        [-0.1467895 ,  0.98770767,  0.05372525, -0.0661803 ],
        [ 0.61185672,  0.04798731,  0.7895116 ,  0.322012  ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.83383763,  0.14465714, -0.5327186 , -0.970912  ],
        [-0.13727125,  0.98907599,  0.05371498, -0.185889  ],
        [ 0.53466943,  0.02833737,  0.84458605,  0.872353  ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.89432262,  0.11451104, -0.43252083, -1.41952   ],
        [-0.10625166,  0.9933957 ,  0.04330777, -0.279885  ],
        [ 0.43462356,  0.00722494,  0.90058324,  1.43657   ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),


In [2]:
read_pose_file("pose.txt")

[array([[ 0.97226635,  0.06500952, -0.22465952, -0.228993  ],
        [-0.06481371,  0.99786324,  0.00825435,  0.00645704],
        [ 0.22471608,  0.00653559,  0.97440236,  0.0287837 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.77722853,  0.14876417, -0.61137961, -0.50237   ],
        [-0.1467895 ,  0.98770767,  0.05372525, -0.0661803 ],
        [ 0.61185672,  0.04798731,  0.7895116 ,  0.322012  ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.83383763,  0.14465714, -0.5327186 , -0.970912  ],
        [-0.13727125,  0.98907599,  0.05371498, -0.185889  ],
        [ 0.53466943,  0.02833737,  0.84458605,  0.872353  ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.89432262,  0.11451104, -0.43252083, -1.41952   ],
        [-0.10625166,  0.9933957 ,  0.04330777, -0.279885  ],
        [ 0.43462356,  0.00722494,  0.90058324,  1.43657   ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),


In [1]:
import numpy as np
import cv2
import open3d as o3d
from scipy.spatial.transform import Rotation as R
import os

def read_pose_file(filename):
    poses = []
    with open(filename, 'r') as f:
        for line in f:
            data = list(map(float, line.strip().split()))
            q = data[3:]  # 四元数部分
            t = data[:3]  # 平移向量部分
            
            # 创建变换矩阵
            T = np.eye(4)
            T[:3, :3] = R.from_quat(q).as_matrix()
            T[:3, 3] = t
            
            poses.append(T)
    return poses

def process_image(color, depth, T, cx, cy, fx, fy, depth_scale):
    points = []
    colors = []
    
    for v in range(color.shape[0]):
        for u in range(color.shape[1]):
            d = depth[v, u]
            if d == 0:
                continue

            z = d / depth_scale
            x = (u - cx) * z / fx
            y = (v - cy) * z / fy

            point = np.dot(T, np.array([x, y, z, 1]))[:3]

            points.append(point)
            colors.append(color[v, u, ::-1] / 255.0)  # BGR to RGB

    return np.array(points), np.array(colors)

def main():
    # 更改工作目录到脚本所在目录
    # os.chdir(os.path.dirname(os.path.abspath(__file__)))

    # 打印当前工作目录
    print("Current working directory:", os.getcwd())

    poses = read_pose_file("/Users/bytedance/Desktop/test/slambook_python/ch5/joinMap/pose.txt")
    print(f"读取了 {len(poses)} 个位姿")

    # 相机内参
    cx, cy = 325.5, 253.5
    fx, fy = 518.0, 519.0
    depth_scale = 1000.0

    # 新建一个点云
    pcd = o3d.geometry.PointCloud()

    for i in range(5):
        try:
            print(f"处理图像 {i+1}")
            color_path = f"/Users/bytedance/Desktop/test/slambook_python/ch5/joinMap/color/{i+1}.png"
            depth_path = f"/Users/bytedance/Desktop/test/slambook_python/ch5/joinMap/depth/{i+1}.pgm"
            
            # if not os.path.exists(color_path) or not os.path.exists(depth_path):
            #     print(f"图像文件不存在: {color_path} 或 {depth_path}")
            #     continue

            color = cv2.imread(color_path)
            depth = cv2.imread(depth_path, -1)  # 以原始深度读取

            if color is None or depth is None:
                print(f"无法读取图像: {color_path} 或 {depth_path}")
                continue

            print(f"颜色图像形状: {color.shape}, 深度图像形状: {depth.shape}")
            print(f"深度图像数据类型: {depth.dtype}")

            points, colors = process_image(color, depth, poses[i], cx, cy, fx, fy, depth_scale)
            
            pcd.points.extend(points)
            pcd.colors.extend(colors)

            print(f"成功处理图像 {i+1}, 当前点云大小: {len(pcd.points)}")

        except Exception as e:
            print(f"处理图像 {i+1} 时发生错误: {str(e)}")

    print(f"点云共有 {len(pcd.points)} 个点.")

    # 保存点云
    o3d.io.write_point_cloud("map.pcd", pcd)

    # 可视化点云
    o3d.visualization.draw_geometries([pcd])

if __name__ == "__main__":
    main()



/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Current working directory: /Users/bytedance/Desktop/test/slambook_python/ch5/joinMap
读取了 5 个位姿
处理图像 1
颜色图像形状: (480, 640, 3), 深度图像形状: (480, 640)
深度图像数据类型: uint16


: 

这里崩溃报错，感觉是点太多了导致的，但是看总的 点数 应该不会呀，，，，有没有大佬能帮看下额。我这里先跳过，后面有时间再看下